In [41]:
import datetime,string,os
import numpy as np
from collections import defaultdict

testset = False
if testset:
	folder = "./test/"
else:
	folder = "./train/"

# get start and enddate of all users
user_dates = {}
with open("./user_dates.csv","r") as fi:
    fi.readline()
    for lines in fi:
        li = lines.strip().split(",")
        user_dates[li[0]] = {'firstdate' : datetime.datetime.strptime(li[1],"%Y-%m-%d").date(),'lastdate' : datetime.datetime.strptime(li[2],"%Y-%m-%d").date() }

# get products we are interested in
productids = set([])
for e, line in enumerate( open("./offers.csv","r") ):
	if e > 0:
		li = line.strip().split(",")
		productids.add(li[1]+" "+li[3]+" "+li[5])
productids = list(productids)  # 優惠券能使用的各商品細項共37種 

with open( os.path.join(folder, "product_repeat_buy_probability.csv"),"w")as of:
    of.write("product_id,repeat_buy_prob_30d,repeat_buy_prob_60d,repeat_buy_prob_90d\n")


    repeaters_30d = defaultdict(list) # 重複購買次數
    repeaters_60d = defaultdict(list)
    repeaters_90d = defaultdict(list)

    with open("./data/reduced.csv","r") as fi:
        fi.readline()
        # set standard variables
        bought_pre_6m = defaultdict(bool)
        firstbuy_date = {}
        repeatbuy_dates = defaultdict(list)
        lastid = 0

        for e,lines in enumerate(fi):
            li = lines.strip().split(",")
            if not lastid == li[0] and e > 0:
                # check if it's valid
                for pi in productids:
                    if pi in firstbuy_date:
                        if (user_dates[user]['lastdate']-firstbuy_date[pi]).days > 30:  # 最後一次消費與第一次購買此商品間隔天數大於30天
                            repeat_within_30d = 0
                            # check if it's repeated within thirty days
                            for d in repeatbuy_dates[pi]:
                                if (d-firstbuy_date[pi]).days < 30:
                                    repeat_within_30d = 1
                            repeaters_30d[pi].append(repeat_within_30d)
                        if (user_dates[user]['lastdate']-firstbuy_date[pi]).days > 60:
                            repeat_within_60d = 0
                            # check if it's repeated within thirty days
                            for d in repeatbuy_dates[pi]:
                                if (d-firstbuy_date[pi]).days < 60:
                                    repeat_within_60d = 1
                            repeaters_60d[pi].append(repeat_within_60d)
                        if (user_dates[user]['lastdate']-firstbuy_date[pi]).days > 90:
                            repeat_within_90d = 0
                            # check if it's repeated within thirty days
                            for d in repeatbuy_dates[pi]:
                                if (d-firstbuy_date[pi]).days < 90:
                                    repeat_within_90d = 1
                            repeaters_90d[pi].append(repeat_within_90d)

                # reset variables
                bought_pre_6m = defaultdict(bool)
                firstbuy_date = {}
                repeatbuy_dates = defaultdict(list)		
            user = li[0]
            product = " ".join(li[3:6])
            date = datetime.datetime.strptime(li[6],"%Y-%m-%d").date()

            if product in productids and float(li[10]) >= 0:
                if (date - user_dates[user]['firstdate']).days < 30*6:
                    # register if user bought the product in first half of year
                    bought_pre_6m[product] = True
                elif bought_pre_6m[product] == False:
                    # candidate for calculation
                    if not product in firstbuy_date:
                        firstbuy_date[product] = date
                    else:
                        # append to dates
                        repeatbuy_dates[product].append(date)
            lastid = li[0]
            if e % 1000000 == 0 and e > 0:
                print(e)

    # get probabilities for categories (replace with this when count is below 100)
    category_repbuy_dict = {}
    with open(os.path.join(folder, "category_repeat_buy_probability.csv"),"r")as fi:
        fi.readline()
        for lines in fi:
            li = lines.strip().split(",")
            category_repbuy_dict[li[0]] = li[1:]

    # write out probabilities
    for pi in productids:
        rep_30 = float(np.sum(repeaters_30d[pi]))/float(len(repeaters_30d[pi]))
        rep_60 = float(np.sum(repeaters_60d[pi]))/float(len(repeaters_60d[pi]))
        rep_90 = float(np.sum(repeaters_90d[pi]))/float(len(repeaters_90d[pi]))
        if len(repeaters_30d[pi]) < 100:
            print("replacing product "+pi+" probs with category probs since number of examples is below 100 :"+str(len(repeaters_30d[pi])))
            cat = pi.split()[0]
            of.write(pi+","+",".join(category_repbuy_dict[cat])+"\n")
        else:
            print("product "+pi+" nums:"+str(len(repeaters_30d[pi])))
            of.write(pi+","+str(rep_30)+","+str(rep_60)+","+str(rep_90)+"\n")          

1000000
2000000
3000000
4000000
5000000
6000000
7000000
8000000
9000000
10000000
11000000
12000000
13000000
14000000
15000000
16000000
17000000
18000000
19000000
20000000
21000000
22000000
23000000
24000000
25000000
26000000
27000000
product 7205 103700030 4294 nums:29644
product 6202 1087744888 64486 nums:2126
product 1726 104460040 7668 nums:836
replacing product 2202 104460040 3718 probs with category probs since number of examples is below 100 :73
replacing product 799 1076211171 17286 probs with category probs since number of examples is below 100 :59
product 5122 107106878 17311 nums:6043
replacing product 5619 107717272 102504 probs with category probs since number of examples is below 100 :4
product 9909 107127979 6732 nums:20478
product 9909 1089520383 28840 nums:1323
product 9115 108500080 93904 nums:559
product 2119 108079383 6926 nums:12709
product 3504 104460040 7668 nums:575
product 1703 104460040 7668 nums:1873
product 4401 105100050 13791 nums:10389
product 5616 1046100